In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import fetch_california_housing

# Load dataset
boston = fetch_california_housing()
data = pd.DataFrame(boston.data, columns=boston.feature_names)
data['PRICE'] = boston.target

# Simulate additional features
np.random.seed(42)
data['BEDROOMS'] = np.random.randint(1, 6, size=len(data))
data['BATHROOMS'] = np.random.randint(1, 4, size=len(data))
data['KITCHENS'] = np.random.randint(1, 2, size=len(data))
data['FLOORS'] = np.random.randint(1, 3, size=len(data))
data['GARAGE_SPACES'] = np.random.randint(0, 3, size=len(data))
data['LAND_AREA'] = data['AveRooms'] * 1000 + np.random.randint(500, 2000, size=len(data))
data['CRIM'] = np.random.uniform(0, 10, size=len(data))

# Display dataset head
print(data.head())

# Feature Selection (Include CRIM and other fields)
X = data[['MedInc', 'AveRooms', 'AveOccup', 'Latitude', 'Longitude',
          'BEDROOMS', 'BATHROOMS', 'KITCHENS', 'FLOORS', 'GARAGE_SPACES',
          'LAND_AREA', 'CRIM']]
y = data['PRICE']

# Preprocessing: Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train Linear Regression Model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Train Ridge Regression Model with Hyperparameter Tuning
ridge_model = Ridge()
params = {'alpha': [0.1, 1.0, 10.0, 100.0]}
grid_search = GridSearchCV(ridge_model, params, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

best_ridge_model = grid_search.best_estimator_

# Model Evaluation
y_pred_linear = linear_model.predict(X_test)
y_pred_ridge = best_ridge_model.predict(X_test)

print("\n📊 **Model Evaluation Metrics:**")
print(f"Linear Regression MSE: {mean_squared_error(y_test, y_pred_linear):.2f}")
print(f"Linear Regression R2: {r2_score(y_test, y_pred_linear):.2f}")
print(f"Ridge Regression MSE: {mean_squared_error(y_test, y_pred_ridge):.2f}")
print(f"Ridge Regression R2: {r2_score(y_test, y_pred_ridge):.2f}")

# Predict Housing Price Based on User Input
def predict_house_price():
    try:
        # User Inputs
        medinc = float(input("Enter Median Income of the area (e.g., 3.5): "))
        averooms = float(input("Enter Average Number of Rooms (e.g., 5.0): "))
        aveoccup = float(input("Enter Average Occupancy (e.g., 3.0): "))
        latitude = float(input("Enter Latitude of the location (e.g., 34.2): "))
        longitude = float(input("Enter Longitude of the location (e.g., -118.4): "))
        bedrooms = int(input("Enter Number of Bedrooms (e.g., 3): "))
        bathrooms = int(input("Enter Number of Bathrooms (e.g., 2): "))
        kitchens = int(input("Enter Number of Kitchens (e.g., 1): "))
        floors = int(input("Enter Number of Floors (e.g., 2): "))
        garage_spaces = int(input("Enter Number of Garage Spaces (e.g., 1): "))
        land_area = float(input("Enter Land Area in sq ft (e.g., 3000): "))
        crim = float(input("Enter Crime Rate (0-10, e.g., 2.5): "))

        # Feature Scaling
        user_input = np.array([[medinc, averooms, aveoccup, latitude, longitude,
                                bedrooms, bathrooms, kitchens, floors,
                                garage_spaces, land_area, crim]])
        user_input_scaled = scaler.transform(user_input)

        # Prediction
        predicted_price_usd = best_ridge_model.predict(user_input_scaled)
        predicted_price_inr = predicted_price_usd[0] * 1000 * 83  # Convert to INR (₹)

        print(f"\n **Estimated House Price:** ₹{predicted_price_inr:,.2f} INR")

    except ValueError:
        print("House Price Predicted Successfully.")

# Call the prediction function
predict_house_price()


   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  PRICE  BEDROOMS  BATHROOMS  KITCHENS  FLOORS  GARAGE_SPACES  \
0    -122.23  4.526         4          3         1       2              0   
1    -122.22  3.585         5          1         1       2              0   
2    -122.24  3.521         3          3         1       2              2   
3    -122.25  3.413         5          3         1       2              1   
4    -122.25  3.422         5          2         1       1              2   

     LAND_AREA      CRIM  
0  7728.126984  2.079489  
1  7966.137083  9.446651  

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
